In [44]:
import sys
sys.path.append('../')
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# import scienceplots
# plt.style.use(['science', 'nature','no-latex'])
import time
import pandas as pd
import traceback
import os

from hamiltonians import Hamiltonian
from qnute_params import QNUTE_params as Params
from qnute_output import QNUTE_output as Output
from qnute import qnute

from helpers import *

# Figures for the paper
## 1. Fidelity to Normalized Time Evolution
This series of figures should explore the fidelity of the simulated time evolution compared to the normalized time evolution (with Taylor series). Each graph should cover a large number of randomized $k$-local Hamiltonians. The lines on the graph show the mean fidelity of the state at time t in the evolution, showing standard deviations at fixed intervals. Different lines correspond to choosing different parameters `D`, `dt`, `trotter_flag`, `taylor_truncate_a`, `taylor_truncate_h`.
For a given qubit lattice and locality $k$, a uniformly random unit-length Hamiltonian is chosen by taking randomly generated $X_I \sim \mathcal{N}(0,1)$ sampled from the standard normal distribution for each $\hat{\sigma}_I$ that can contribute to a $k$-local Hamiltonian, and normalizing the random vector $Y_I = \frac{X_I}{\sum_I X_I^2}$, and lastly generating uniform random phases $\Phi_I \sim U(0,2\pi)$ giving the coefficients $e^{i\Phi_I}Y_I$.

The graphs will be Fidelity (y-axis) vs Simulation time (x-axis) line graphs with different colors corresponding to different parameters chosen.

## 2. Fidelity variation due to measurement samples
This series of figures shows how changing the number of measurement samples in the QNUTE simulation to generate the system of linear equations in each Trotter step affects the fidelity of the simulation when compared to taking the theoretical expected values with knowledge of the state vector.

As before, each plot corresponds to a different Hamiltonian structure. We will also separate plots by the different run parameters to see if they affect the fidelity.

These should be mulit-bar graphs with error bars. Each set of bars corresponds to a number of measurement samples and the different bars in the set indicate the different run parameters, along with an aggregate over all the experiments.

Fidelity (y-axis) vs Number of Samples (x-axis, log scale)

### Generating Random k-local Hamiltonians
Ideas for how to find all possible connected paths of length $k$ in the lattice of $N$ points.
1. Loop through all possibilities of $k$ points and see if they form a connected path. $O(N^k)$ Very inneficient.
2. Loop through every point and recursively build all possible paths of length k starting from that point. $O(N2^{d(k-1)})$

Coordinate ordering by index i.e. $(0,0)\prec(0,1)\prec(1,0)\prec(1,1)$

In an unbounded grid, there will be a total of $1\cdot2^d\cdot(2^d-1)^{k-1} = O(2^{dk})$ paths of length $k>0$ originating from a fixed starting point that don't self intersect. A path of length $k$ has $k+1$ vertices in it.

In [112]:
def get_paths(start, depth,d,l, illegal_dir=None):
    '''
    returns all paths of depth=depth originating from the point start in the l^d lattice using depth first search
    the paths are sorted coordinate wise
    '''
    if type(start) is not tuple:
        start = (start, )
    assert len(start) == d, 'start must be a d-dimensional tuple'
    
    if depth == 0:
        return [[start]]  if d > 1 else [[start[0]]]
    
    paths = []
    for i in range(d):
        for j in range(2):
            if illegal_dir == (i,(-1)**j):
                continue
            n_node = list(start)
            n_node[i] += (-1)**j
            n_node = tuple(n_node)
            if in_lattice(n_node,d,l):
                n_paths = get_paths(n_node, depth-1, d,l, illegal_dir=(i,(-1)**(j+1)))
                for path in n_paths:
#                     paths.append( [start]+path )
                    paths.append( [start if d > 1 else start[0]]+path )
                    
    return paths

def get_k_local_domains(k,d,l):
    domains = set()
    for i in range(l**d):
        point = tuple(int_to_base(i,l,d))
        for k_ in range(k):
            n_paths = get_paths(point,k_,d,l)
#             print('point:{}, k={}, paths={}'.format(point, k_, n_paths))
            for path in n_paths:
                s_path = sorted(path)
                domains.add(tuple(s_path))
    return sorted(list(domains))

def get_random_complex_vector(n):
    '''
    generates a uniformly random complex vector in C^n
    '''
    x = np.random.normal(size=n)
    x /= np.linalg.norm(x)
    phi = np.random.uniform(low=-np.pi, high=np.pi,size=n)
    return np.exp(1j*phi)*x

def get_random_k_local_hamiltonian(k, d, l, qubit_map, domains=None):
    if domains is None:
        domains = get_k_local_domains(k, d, l)
    
    hm_list = [ [ [], np.zeros(3**len(domain),dtype=complex), list(domain) ] for domain in domains ]
    num_coeffs = 0
    for hm in hm_list:
        ndomain = len(hm[2])
        num_coeffs += 3**ndomain
        for p in range(3**ndomain):
            pstring = int_to_base(p, 3, ndomain)
            for i in range(ndomain):
                pstring[i] += 1
            hm[0].append(base_to_int(pstring, 4))
    coeffs = get_random_complex_vector(num_coeffs)
    start = 0
    for hm in hm_list:
        hm[1] = coeffs[start:start+len(hm[1])]
        start += len(hm[1])
    
    return Hamiltonian(hm_list, d, l, qubit_map)

def save_numerical_evolution(H_mat, psi0, dt, N, path:str):
    times = np.arange(0,N+1,1)*dt
    svs = np.zeros((N+1, psi0.shape[0]),dtype=complex)
    for i in range(len(times)):
        svs[i] = exp_mat_psi(H_mat*times[i], psi0)
        svs[i] /= np.linalg.norm(svs[i])
    rdf = pd.DataFrame(np.real(svs))
    idf = pd.DataFrame(np.imag(svs))
    rdf.insert(0,'t',times)
    idf.insert(0,'t',times)
    rdf = rdf.sort_values('t')
    idf = idf.sort_values('t')
    
    if not os.path.exists(path):
        os.makedirs(path)
    rdf.to_csv(path+'num_statevectors_real.csv',index=False)
    idf.to_csv(path+'num_statevectors_imag.csv',index=False)
    return svs

# Generating the Random Hamiltonians and Running the QNUTE experiments

In [156]:
def get_statevectors_from_csv(file):
    rdf = pd.read_csv(file+'_real.csv')
    idf = pd.read_csv(file+'_imag.csv')
    return rdf.drop('t',axis=1).to_numpy() + 1.0j * idf.drop('t',axis=1).to_numpy()

In [160]:
num_expts = 1
digits = int(np.floor(np.log10(num_expts))+1)
if digits < 3:
    digits = 3

# What file to log to
t = time.localtime()
# current_time = time.strftime('%Y-%m-%d-%H-%M-%S',t)
current_time = time.strftime('%Y-%m-%d',t)
log_path = './logs/{}/'.format(current_time)

log_file = 'test.log'

if not os.path.exists(log_path):
    os.makedirs(log_path)
    
original_stdout = sys.stdout

separator = '------\n'

# Hamiltonian Properties
k=2
d=1
l=2
qubit_map=None
domains = get_k_local_domains(k,d,l)

# QNUTE Parameters
# List containing the different parameters to loop through for each random Hamiltonian

# Unitary domain diameters
Ds=[1,2]

# Total simulated time T=N*dt
T = 1.0
dts = np.array([0.1, 0.05, 0.01])
Ns=np.ceil(T/dts).astype(int)

# Regularizer
delta = 0.1

# Number of measurement samples
num_shots = [0]

# Use a taylor series to calculate the norm c= || e^{h_m dt}|psi> ||
taylor_norm_flag=False
taylor_truncate_h=-1

# When to truncate the Taylor series for the unitary updates
taylor_truncate_a=-1
# Whether to use a Trotterized update or not
trotter_flags=[False, True]

backend=None
init_sv = np.zeros(2**(l**d),dtype=complex)
init_sv[0] = 1.0
objective_meas_list = [ [[1,3], [0,1]] ]

f = open(log_path+log_file,'w')
try:
    # redirect all print statements to the log file
    sys.stdout = f

    print('Total number of experiments:', num_expts)
    print('k={}, lattice_dim={}, lattice_bound={}'.format(k,d,l))
    print('Qubit mapping:')
    if qubit_map is None:
        print('\tDefault 1-D mapping')
    else:
        for key in qubit_map.keys():
            print('\t{} -> {},'.format(key, qubit_map[key]))
    if objective_meas_list is None or len(objective_meas_list) == 0:
        print('No objective measurements')
    else:
        print('List of objective measurements:')
        for m_list in objective_meas_list:
            qbits = m_list[1]
            for p in m_list[0]:
                pstring = int_to_base(p,4,len(qbits))
                m_name = ''
                for i in range(len(qbits)):
                    if pstring[i] == 0: m_name += 'I'
                    else: m_name += chr(ord('X')+pstring[i]-1)
                    m_name += '_'
                    m_name += str(qbits[i])
                    if i < len(qbits) - 1: m_name += ' '
                print('\t{},'.format(m_name))

    print(separator)

    for i in range(num_expts):
        t1 = time.localtime()
        print('Started Experiment #{} at {}'.format(format(i+1,'0{}d'.format(digits)), time.strftime('%I:%M:%S %p, %d %b %Y', t1)))
        expt_path = log_path + 'expt_{}/'.format(format(i+1,'0{}d'.format(digits)))
        if not os.path.exists(expt_path):
            os.makedirs(expt_path)

        H = get_random_k_local_hamiltonian(k,d,l,qubit_map,domains)
        print('Randomly Generated Effective Hamiltonian:')
        H.print()
        H.to_csv(expt_path+'ham.csv')
        print('Saved Hamiltonian description in \'{}ham.csv\''.format(expt_path))
        print(separator)

        params = Params(H)
        for t_i in range(len(dts)):
            dt = dts[t_i]
            N  = Ns[t_i]
            t = np.arange(0,N+1,1)*dt
            print('Numerically calculating normalized non-unitary evolution')
            num_svs = save_numerical_evolution(H.get_matrix(), init_sv, dt, N, expt_path+'/dt={}/'.format(dt))
            print(separator)
            for D in Ds:
                params.load_hamiltonian_params(D,False,True)
                for trotter in trotter_flags:
                    for shots in num_shots:
                        print('Setting QNUTE run parameters:')
                        print('\tD={}'.format(D))
                        print('\tdt={}'.format(dt))
                        print('\tdelta={}'.format(delta))
                        print('\tN={}'.format(N))
                        print('\tnum_shots={}'.format(shots))
                        print('\ttaylor_norm_flag={}'.format(taylor_norm_flag))
                        print('\ttaylor_truncate_h={}'.format(taylor_truncate_h))
                        print('\ttaylor_truncate_a={}'.format(taylor_truncate_a))
                        print('\ttrotter_flag={}'.format(trotter))
                        params.set_run_params(dt,delta,N,shots,backend,
                                              init_sv=init_sv,
                                              taylor_norm_flag=taylor_norm_flag, 
                                              taylor_truncate_h=taylor_truncate_h, 
                                              taylor_truncate_a=taylor_truncate_a, 
                                              trotter_flag=trotter,
                                              objective_meas_list=objective_meas_list)
                        instance_path = expt_path + 'dt={}/D={}/shots={}/trotter={}/'.format(dt,D,shots,trotter)
                        if not os.path.exists(instance_path):
                            os.makedirs(instance_path)

                        print('Started QNUTE instance at', time.strftime('%I:%M:%S %p, %d %b %Y',time.localtime()))
                        output = qnute(params, log_to_console=True)
                        print('Finished QNUTE instance at', time.strftime('%I:%M:%S %p, %d %b %Y',time.localtime()))

                        output.log_output('run', path=instance_path)
                        print('Saved QNUTE outputs in \'{}\''.format(instance_path))
                        
                        print('Calculating fidelities')
                        fids = np.abs(np.diag( num_svs.conj() @ output.svs.T ))
                        fid_df = pd.DataFrame(data = {'t':t, 'Fidelity': fids})
                        fid_df.to_csv(instance_path+'fidelity_to_taylor.csv',index=False)
                        
                        if shots == 0:
                            s0_svs = get_statevectors_from_csv(instance_path+'run_statevectors')
                        if shots > 0:
                            # Compare to the shots=0 run with the same parameters
                            fids = np.abs(np.diag( s0_svs.conj() @ output.svs.T ))
                            fid_df = pd.DataFrame(data = {'t':t, 'Fidelity': fids})
                            fid_df.to_csv(instance_path+'fidelity_to_0_shots.csv',index=False)
                        print(separator)

        t2 = time.localtime()
        print('Finished Experiment #{} at {}'.format(format(i+1,'0{}d'.format(digits)), time.strftime('%I:%M:%S %p, %d %b %Y', t2)))
        print(separator)

    print('Finished execution at {}'.format(time.strftime('%I:%M:%S %p, %d %b %Y', time.localtime())))
except Exception as e:
    print(separator)
    print('{}: {}'.format(type(e).__name__, e))
    print(traceback.format_exc())
    t = time.localtime()
    print('Terminating run at', time.strftime('%I:%M:%S %p, %d %b %Y', t))
finally:
    # reset stdout to the console
    sys.stdout = original_stdout
    f.close()

C:\Users\swaga\Documents\QNUTE\Sandbox Notebooks\..\helpers.py:80: RuntimeWarning: invalid value encountered in double_scalars
  return np.sum(points,axis=0)/n
C:\Users\swaga\Documents\QNUTE\Sandbox Notebooks\..\helpers.py:80: RuntimeWarning: invalid value encountered in double_scalars
  return np.sum(points,axis=0)/n
C:\Users\swaga\Documents\QNUTE\Sandbox Notebooks\..\helpers.py:80: RuntimeWarning: invalid value encountered in double_scalars
  return np.sum(points,axis=0)/n
C:\Users\swaga\Documents\QNUTE\Sandbox Notebooks\..\helpers.py:80: RuntimeWarning: invalid value encountered in double_scalars
  return np.sum(points,axis=0)/n
C:\Users\swaga\Documents\QNUTE\Sandbox Notebooks\..\helpers.py:80: RuntimeWarning: invalid value encountered in double_scalars
  return np.sum(points,axis=0)/n
C:\Users\swaga\Documents\QNUTE\Sandbox Notebooks\..\helpers.py:80: RuntimeWarning: invalid value encountered in double_scalars
  return np.sum(points,axis=0)/n


In [157]:
s = instance_path
s.replace('')

'./logs/2023-02-21/expt_001/dt=0.01/D=2/shots=0/trotter=True/'

# Using the logged data to generate the fidelity graphs: